In [ ]:
import os
import pydub
import requests
import pickle

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scipy.io import wavfile
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Data Collection & Processing

#### Note on Data Collection:

Had we been able to acquire an API token for streaming live audio from [Broadcastify](https://www.broadcastify.com/) (ostensibly they are concerned with mobile app developers abusing their service) our first step documented here would be API calls retrieving either archival or streaming radio feeds. Prevented from doing so, we elected to manually download the archives from all the feeds proximate in time and space to a recent natural disaster. Given that Broadcastify's archives only go back 180 days, we chose the recent catastrophic Camp Fire of last fall. From the seven feeds in Butte County, California we downloaded the nearly complete 24 hours of radio from the period of November 8 to November 23 of 2018 (the official period of the Camp Fire). In sum this provided us with 3,085 half-hour long MP3 files (13.8 GB).

## Audio Signal Processing

Scanner radio is always "on", even when there is no transmission. This means that each half-hour MP3 we collected was mostly radio silence. Also, each transmission is punctuated with the expected radio buzz and click noises. As-is the MP3 files were far from optimal samples for our model. They required cleaning and shaping. We used a library, [Pydub](http://pydub.com/), to perform our basic audio signal processing. This we did with 2 requirements in mind:

    1. conforming to the specifications of Google Cloud Speech-to-Text API
    2. concentration on the speech transmitted
    
Google Cloud's transcription service does not receive MP3 files, and has several additional specifications for the audio it can transcribe (channel count, sample width, length, size, sample rate, etc.). We therefore had to convert our file to the WAV format, and ensure we were meeting these criteria. Also, for the sake of accuracy, we wanted to eliminate the noises that were not speech, and isolate in short phrases the actual content of the transmissions.

In [35]:
sound_file = AudioSegment.from_file("./audio/201811080143-801680-26936 (1).mp3", format="mp3")
channel_count = sound_file.channels    
sample_width = sound_file.sample_width 
duration_in_sec = len(sound_file) / 1000 
sample_rate = sound_file.frame_rate

print ("sample_width=", sample_width)
print ("channel_count=", channel_count)
print ("duration_in_sec=", duration_in_sec)
print ("frame_rate=", sample_rate)

sample_width= 2
channel_count= 1
duration_in_sec= 1812.0
frame_rate= 48000


Above we performed checks on the specifications of our audio files and determined them as fit for splitting.

Below we use methods from Pydub (AudioSegment) to eliminate and split on the silence in our MP3 files and return the segments as WAV files. We experimented with several values for the two parameters `min_silence_len` and `silence_thresh` and found defining silence as 1 second long and anything lower than -30 decibels was best. If any segment (in the code `chunks`) was either less than 8.5 seconds or more than 60 seconds long we did not export it to our `/samples` directory. In this way we were able to eliminate most of the radio noises, the radio silence, and generate nearly 100,000 speech samples 1 to 60 seconds in length (20 GB). 


In [ ]:
for filename in os.listdir('./audio'):
    if filename.endswith('.mp3'):
        file_name = './audio/' + filename
        sound_file = AudioSegment.from_file(file_name, format="mp3")
        chunks = split_on_silence(sound_file, 
                                  min_silence_len = 1000,            
                                  silence_thresh = -30)              

        for i, chunk in enumerate(chunks):
            if (len(chunk)) > 1000 | (len(chunk)) > 60000:
                chunk.export(f"./samples_1/{filename[:12]}_{i}.wav", format="wav")
            else:
                pass
    else:
        pass

## Webscraping Vocabulary Data

In anticipation of the services we will be using, and knowing that we will not be supervising the model, we found and scraped an exhaustive list of road names in the same (Butte) county that we collected radio scanner feeds from. Both the speech recognition and natural language processing models we use will allow us to enter a custom vocabulary list to assist in the recognition and extraction of the listed words. We found this list at [geographic.org](https://geographic.org/) and scraped the contents of the webpage using the [Beautiful Soup](https://pypi.org/project/beautifulsoup4/) parsing library.

In [ ]:
url = 'https://geographic.org/streetview/usa/ca/butte/'
res = requests.get(url)
res.status_code

In [ ]:
soup = BeautifulSoup(res.content, 'lxml')

list_of_cities = []
list_of_roads = []
body = soup.find('ul')
for city in body.find_all('li'):
    list_of_cities.append(city.text)
    
for i in range(len(list_of_cities)):
    list_of_cities[i] = list_of_cities[i].lower()
    list_of_cities[i] = list_of_cities[i].replace(' ', '_')
    
for city in list_of_cities:
    url = 'https://geographic.org/streetview/usa/ca/butte/' + city + '.html'
    print('now scraping', url)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    body= soup.find('ul')
    for road in body.find_all('li'):
        list_of_roads.append(road.text)
        
for i in range(len(list_of_roads)):
    list_of_roads[i] = list_of_roads[i][slice(-8)]
    
len(list_of_roads)

Our `soup` object is directed to the `url` where it loops through every city in a `list_of_cities` where it parses and appends each `li` road name to a `list_of_roads`. We then save the variable `list_of_roads` as an environment variable using `pickle` in order to pass it on to our other jupyter notebooks.

In [ ]:
with open("list_of_roads.txt", "wb") as fp:
    pickle.dump(list_of_roads, fp)